jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [151]:
import pandas as pd
from tqdm import tqdm
import time
import os

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [152]:
url = "https://race.netkeiba.com/race/result.html?race_id=202209011101"

pd.set_option('display.max_rows',20)
pd.options.display.width = 200
pd.options.display.column_space = 100

# pd.read_html(url)[0]


レース結果をスクレイピング

In [153]:
if(os.path.exists('results.pickle')):
  race_result = pd.read_pickle("results.pickle")
else:
  with open("results.pickle","w"):pass

In [154]:
race_id_lists = []

for 年 in range(2021,2022):
  for 競馬場 in range(1,5): # 1,11
    for 回 in range(1,4): # 1,11
      for 回目 in range(1,5): # 1,11
        for レース回 in range(1,3): # 1,13
          race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
          race_id_lists.append(race_id)
# print(race_id_lists)

# 実行時間 72分
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    if _race_id in race_result.keys():
      continue
    time.sleep(1)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict
  
race_result = scrape_race_results(race_id_lists)

100%|██████████| 96/96 [02:12<00:00,  1.38s/it]


In [155]:
race_result = scrape_race_results(race_result)

100%|██████████| 64/64 [00:00<00:00, 1335499.78it/s]


In [156]:
len(race_result)

64

In [157]:

race_result

{'202101010101':     着順  枠  馬番         馬名  性齢    斤量    騎手     タイム     着差  人気  単勝オッズ   後3F コーナー通過順     厩舎   馬体重(増減)
 0    1  5   9  ディーバサンライズ  牝3  52.0  △秋山稔  1:08.7    NaN   7   13.5  34.6     4-4   栗東小林  474(-22)
 1    2  3   6      バンベルク  牡3  56.0   横山和  1:08.8    3/4   2    3.9  35.0     1-1  栗東安田隆   470(-2)
 2    3  7  14      リツィタル  牝3  54.0    大野  1:08.9    1/2  14  124.7  34.0   11-10   栗東牧田   394(+2)
 3    4  5  10  クレマチステソーロ  牝3  54.0   鮫島駿  1:09.0    1/2   5    7.6  35.2     2-2   美浦高木   468(+2)
 4    5  3   5  ペイシャケイティー  牝3  54.0   古川吉  1:09.0     ハナ   4    7.5  34.5     7-7   美浦土田   456(+8)
 5    6  1   2       セリシア  牝3  54.0    武豊  1:09.0     クビ   1    3.7  34.9     4-4   栗東中竹   456(+4)
 6    7  4   7      デクラーク  牡3  56.0   横山武  1:09.2      1   3    6.1  33.9   15-15   美浦矢野   444(+2)
 7    8  6  11    プレジャジンク  牝3  54.0   吉田隼  1:09.2    アタマ   8   25.0  35.3     3-2   栗東吉田  452(+12)
 8    9  7  13     メルティング  牝3  54.0    団野  1:09.6  2.1/2   9   56.6  34.5   12-12  栗東池添兼    4

dictのkeyをrace_idに変換


In [158]:
for key in race_result.keys():
  race_result[key].index = [key] * len(race_result[key])
print(type(race_result))

<class 'dict'>


全てのdictを結合し、dataframeに変換

In [159]:
results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
print(type(results))

<class 'pandas.core.frame.DataFrame'>


In [160]:
results.to_pickle("results.pickle")

In [161]:
time.sleep(2)

In [162]:
# pd.read_pickle("results.pickle")

,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減),Unnamed: 15,Unnamed: 16,Unnamed: 17
202101010101,1,5,9,ディーバサンライズ,牝3,52.0,△秋山稔,1:08.7,NaN,7,13.5,34.6,4-4,栗東小林,474(-22),NaN,NaN,NaN
202101010101,2,3,6,バンベルク,牡3,56.0,横山和,1:08.8,3/4,2,3.9,35.0,1-1,栗東安田隆,470(-2),NaN,NaN,NaN
202101010101,3,7,14,リツィタル,牝3,54.0,大野,1:08.9,1/2,14,124.7,34.0,11-10,栗東牧田,394(+2),NaN,NaN,NaN
202101010101,4,5,10,クレマチステソーロ,牝3,54.0,鮫島駿,1:09.0,1/2,5,7.6,35.2,2-2,美浦高木,468(+2),NaN,NaN,NaN
202101010101,5,3,5,ペイシャケイティー,牝3,54.0,古川吉,1:09.0,ハナ,4,7.5,34.5,7-7,美浦土田,456(+8),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202104030402,14,2,3,クラーラ,牝2,54.0,岩部,0:58.5,2,11,111.0,35.0,12,美浦奥平,432(-12),NaN,NaN,NaN
202104030402,15,1,1,フェスティヴメノコ,牝2,54.0,西村淳,0:58.9,2.1/2,12,116.4,35.6,8,美浦清水英,420(+8),NaN,NaN,NaN
202104030402,16,7,13,ブルーアンドグレイ,牡2,54.0,長岡,0:59.3,2,17,228.4,34.9,17,美浦加藤和,438(-2),NaN,NaN,NaN
202104030402,17,3,5,ピンクパフェ,牝2,54.0,嘉藤,0:59.6,2,18,304.0,35.8,14,美浦宗像,414(-2),NaN,NaN,NaN
